## **Question Answering with BERT (SQuAD Dataset)**

### **Objective:**

Teach learners how BERT can be fine-tuned for **Question Answering (QA)** tasks using the SQuAD dataset. By the end, learners will understand:

* How QA datasets are structured
* How BERT tokenizes and handles input
* How to fine-tune BERT for QA
* How to predict answers to new questions

### **Step 1: Install Dependencies**

In [1]:
!pip install transformers datasets torch

### **Step 2: Import Libraries**

In [2]:
import torch
from transformers import BertTokenizerFast, BertForQuestionAnswering
from datasets import load_dataset

### **Step 3: Load the SQuAD Dataset**

In [3]:
# Load SQuAD v1.1 dataset
dataset = load_dataset("squad")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [4]:
# Check the first example
print(dataset['train'][0])

{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}


* `context`: Paragraph where the answer is located
* `question`: The question we want to answer
* `answers`: Target answer(s) with start positions in `context`

### **Step 4: Initialize BERT Tokenizer and Model**

In [5]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

In [6]:
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


> Note: We use a **pre-finetuned BERT model on SQuAD** so learners can experiment immediately without heavy training.

### **Step 5: Encode Inputs for BERT**

In [7]:
def encode_example(example):
    return tokenizer(
        example['question'],
        example['context'],
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors="pt"
    )

In [8]:
example = dataset['train'][0]
inputs = encode_example(example)
inputs.keys()

KeysView({'input_ids': tensor([[  101,  2000,  3183,  2106,  1996,  6261,  2984,  9382,  3711,  1999,
          8517,  1999, 10223, 26371,  2605,  1029,   102,  6549,  2135,  1010,
          1996,  2082,  2038,  1037,  3234,  2839,  1012, 10234,  1996,  2364,
          2311,  1005,  1055,  2751,  8514,  2003,  1037,  3585,  6231,  1997,
          1996,  6261,  2984,  1012,  3202,  1999,  2392,  1997,  1996,  2364,
          2311,  1998,  5307,  2009,  1010,  2003,  1037,  6967,  6231,  1997,
          4828,  2007,  2608,  2039, 14995,  6924,  2007,  1996,  5722,  1000,
          2310,  3490,  2618,  4748,  2033, 18168,  5267,  1000,  1012,  2279,
          2000,  1996,  2364,  2311,  2003,  1996, 13546,  1997,  1996,  6730,
          2540,  1012,  3202,  2369,  1996, 13546,  2003,  1996, 24665, 23052,
          1010,  1037, 14042,  2173,  1997,  7083,  1998,  9185,  1012,  2009,
          2003,  1037, 15059,  1997,  1996, 24665, 23052,  2012, 10223, 26371,
          1010,  2605,  2073,

**Key outputs:**

* `input_ids`: Token IDs
* `attention_mask`: Mask to ignore padding tokens
* `token_type_ids`: Distinguishes question tokens vs context tokens

### **Step 6: Make Predictions**

In [9]:
with torch.no_grad():
    outputs = model(**inputs)

In [10]:
start_scores = outputs.start_logits
end_scores = outputs.end_logits

In [11]:
# Get the most likely start and end tokens
start_idx = torch.argmax(start_scores)
end_idx = torch.argmax(end_scores)

In [12]:
# Decode answer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(
    inputs['input_ids'][0][start_idx:end_idx+1]
))

In [13]:
print("Question:", example['question'])
print("Predicted Answer:", answer)
print("Actual Answer:", example['answers']['text'][0])

Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Predicted Answer: saint bernadette soubirous
Actual Answer: Saint Bernadette Soubirous


### **Step 7: Test with Your Own Question**

In [14]:
context = "The Eiffel Tower is a wrought-iron lattice tower in Paris, France."
question = "Where is the Eiffel Tower located?"

In [15]:
inputs = tokenizer(question, context, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

In [16]:
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits)
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx+1])
)

In [17]:
print("Question:", question)
print("Predicted Answer:", answer)

Question: Where is the Eiffel Tower located?
Predicted Answer: paris, france
